In [13]:
import pandas as pd

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [17]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder

X_train = train_data.drop('Activity', axis=1)
y_train = train_data['Activity']
X_test = test_data.drop('Activity', axis=1)
y_test = test_data['Activity']

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [18]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=100)
X_train_selected = selector.fit_transform(X_train, y_train_encoded)
X_test_selected = selector.transform(X_test)

## Classical Machine Learning Models

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_selected, y_train)
y_pred = rf.predict(X_test_selected)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

Accuracy: 0.8870037326094333


## Deep Learning Models

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten

# Reshape data for CNN
X_train_cnn = X_train_selected.reshape(X_train_selected.shape[0], X_train_selected.shape[1], 1)
X_test_cnn = X_test_selected.reshape(X_test_selected.shape[0], X_test_selected.shape[1], 1)

model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_selected.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_cnn, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test_cnn, y_test_encoded))

Epoch 1/10
230/230 [==============================] - 2s 8ms/step - loss: 0.4642 - accuracy: 0.7881 - val_loss: 0.2923 - val_accuracy: 0.8853
Epoch 2/10
230/230 [==============================] - 2s 7ms/step - loss: 0.2765 - accuracy: 0.8757 - val_loss: 0.3173 - val_accuracy: 0.8541
Epoch 3/10
230/230 [==============================] - 2s 7ms/step - loss: 0.2443 - accuracy: 0.8889 - val_loss: 0.2361 - val_accuracy: 0.8918
Epoch 4/10
230/230 [==============================] - 2s 7ms/step - loss: 0.2119 - accuracy: 0.9078 - val_loss: 0.2279 - val_accuracy: 0.8911
Epoch 5/10
230/230 [==============================] - 2s 7ms/step - loss: 0.2047 - accuracy: 0.9091 - val_loss: 0.2060 - val_accuracy: 0.9070
Epoch 6/10
230/230 [==============================] - 1s 6ms/step - loss: 0.1980 - accuracy: 0.9120 - val_loss: 0.2152 - val_accuracy: 0.9091
Epoch 7/10
230/230 [==============================] - 2s 7ms/step - loss: 0.1881 - accuracy: 0.9185 - val_loss: 0.2283 - val_accuracy: 0.9009
Epoch 

In [21]:
from sklearn.metrics import classification_report

y_pred_dl = model.predict(X_test_cnn)
y_pred_dl_classes = y_pred_dl.argmax(axis=1)

print(classification_report(y_test_encoded, y_pred_dl_classes, target_names=label_encoder.classes_))

93/93 [==============================] - 0s 3ms/step
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.91      0.66      0.76       491
          STANDING       0.75      0.94      0.83       532
           WALKING       0.91      0.99      0.95       496
WALKING_DOWNSTAIRS       0.96      0.95      0.95       420
  WALKING_UPSTAIRS       0.96      0.88      0.92       471

          accuracy                           0.90      2947
         macro avg       0.91      0.90      0.90      2947
      weighted avg       0.91      0.90      0.90      2947



In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, GRU, Dropout, BatchNormalization
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

X = train_data.drop('Activity', axis=1)
y = train_data['Activity']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape data for CNN
X_train_cnn = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build hybrid model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    LSTM(50, return_sequences=True),
    GRU(50),
    Dropout(0.5),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_cnn, y_train, epochs=20, batch_size=32, validation_data=(X_test_cnn, y_test))

# Evaluate model
y_pred = model.predict(X_test_cnn)
y_pred_classes = y_pred.argmax(axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))

2024-06-02 23:23:32.520426: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-02 23:23:32.521820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-02 23:23:32.522954: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/20


2024-06-02 23:23:33.085315: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-02 23:23:33.087693: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-02 23:23:33.089071: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

184/184 [==============================] - ETA: 0s - loss: 0.9304 - accuracy: 0.5531

2024-06-02 23:24:11.378437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-02 23:24:11.381197: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-02 23:24:11.383381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

184/184 [==============================] - 42s 206ms/step - loss: 0.9304 - accuracy: 0.5531 - val_loss: 0.8842 - val_accuracy: 0.6390
Epoch 2/20
184/184 [==============================] - 44s 239ms/step - loss: 0.5213 - accuracy: 0.7776 - val_loss: 0.5235 - val_accuracy: 0.7967
Epoch 3/20
184/184 [==============================] - 45s 244ms/step - loss: 0.3765 - accuracy: 0.8497 - val_loss: 0.3359 - val_accuracy: 0.8498
Epoch 4/20
184/184 [==============================] - 44s 238ms/step - loss: 0.3122 - accuracy: 0.8830 - val_loss: 0.3387 - val_accuracy: 0.8640
Epoch 5/20
184/184 [==============================] - 43s 234ms/step - loss: 0.2709 - accuracy: 0.9017 - val_loss: 0.2440 - val_accuracy: 0.9028
Epoch 6/20
184/184 [==============================] - 48s 258ms/step - loss: 0.2311 - accuracy: 0.9141 - val_loss: 0.2938 - val_accuracy: 0.8899
Epoch 7/20
184/184 [==============================] - 46s 248ms/step - loss: 0.2293 - accuracy: 0.9191 - val_loss: 0.1853 - val_accuracy: 0.9

2024-06-02 23:38:04.269436: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-02 23:38:04.275364: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-02 23:38:04.278068: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

46/46 [==============================] - 5s 79ms/step
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       280
           SITTING       0.90      0.87      0.89       262
          STANDING       0.88      0.91      0.90       276
           WALKING       0.98      0.98      0.98       247
WALKING_DOWNSTAIRS       0.98      0.99      0.98       206
  WALKING_UPSTAIRS       0.98      0.97      0.97       200

          accuracy                           0.95      1471
         macro avg       0.95      0.95      0.95      1471
      weighted avg       0.95      0.95      0.95      1471



## transfer learning

In [25]:
# Build hybrid model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    LSTM(50, return_sequences=True),
    GRU(50),
    Dropout(0.5),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_cnn, y_train, epochs=20, batch_size=32, validation_data=(X_test_cnn, y_test))

2024-06-02 23:42:00.262664: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-02 23:42:00.265970: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-02 23:42:00.267662: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/20


2024-06-02 23:42:00.966311: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-02 23:42:00.968165: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-02 23:42:00.969729: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

184/184 [==============================] - ETA: 0s - loss: 0.8941 - accuracy: 0.5730

2024-06-02 23:42:47.710434: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-02 23:42:47.712149: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-02 23:42:47.713928: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

184/184 [==============================] - 50s 239ms/step - loss: 0.8941 - accuracy: 0.5730 - val_loss: 1.0795 - val_accuracy: 0.4861
Epoch 2/20
184/184 [==============================] - 45s 246ms/step - loss: 0.5050 - accuracy: 0.7820 - val_loss: 0.8146 - val_accuracy: 0.6404
Epoch 3/20
184/184 [==============================] - 43s 232ms/step - loss: 0.3540 - accuracy: 0.8643 - val_loss: 0.4633 - val_accuracy: 0.8226
Epoch 4/20
184/184 [==============================] - 45s 246ms/step - loss: 0.3023 - accuracy: 0.8825 - val_loss: 0.2872 - val_accuracy: 0.8960
Epoch 5/20
184/184 [==============================] - 44s 237ms/step - loss: 0.2589 - accuracy: 0.9007 - val_loss: 0.2433 - val_accuracy: 0.9116
Epoch 6/20
184/184 [==============================] - 44s 237ms/step - loss: 0.2310 - accuracy: 0.9143 - val_loss: 0.2764 - val_accuracy: 0.8953
Epoch 7/20
184/184 [==============================] - 44s 238ms/step - loss: 0.2200 - accuracy: 0.9158 - val_loss: 0.2238 - val_accuracy: 0.8

In [26]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cnn)
y_pred_classes = y_pred.argmax(axis=1)

print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))

2024-06-02 23:56:41.166383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-02 23:56:41.167877: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-02 23:56:41.169095: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

46/46 [==============================] - 4s 60ms/step
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       280
           SITTING       0.88      0.90      0.89       262
          STANDING       0.91      0.88      0.89       276
           WALKING       0.98      0.98      0.98       247
WALKING_DOWNSTAIRS       0.97      0.95      0.96       206
  WALKING_UPSTAIRS       0.92      0.97      0.95       200

          accuracy                           0.94      1471
         macro avg       0.94      0.94      0.94      1471
      weighted avg       0.94      0.94      0.94      1471



In [ ]:
## voting 
